In [1]:
# General purpose
import pandas as pd
import os
import glob
import json
import numpy as np
import string
import matplotlib.pyplot as plt
from datetime import datetime
from ast import literal_eval
#word2vec
from gensim.models.word2vec import Word2Vec
#sklearn related
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.model_selection import cross_val_score


#setting the correct working directory
os.chdir("/home/willinki/DATA-SCIENCE-1/data-semantics/Just-News/1_ScoreInduction")

# Open the pre-prepared lexicon
with open("../lexicon/enriched_lexicon.csv", 'r') as file:
    enriched_lexicon = pd.read_csv(file, 
                                   engine='c', 
                                   index_col=0)
enriched_lexicon["Vectorized_words"] = enriched_lexicon["Vectorized_words"].apply(literal_eval)

#loading models and selecting
slices = {
          filename.split('/')[-1].replace(".model", "") : 
          Word2Vec.load(filename)
          for filename in glob.glob('../models/*.model')
         }

##########################################################
# SELECT WHICH MODELS YOU WANT TO USE                    #
##########################################################
model_names = ["New York Times", "Breitbart"]
##########################################################

models = [slices[name] for name in model_names]

# Score induction: Hamilton method

We have to define an embedding first

In [2]:
import embedding
import polarity_induction_methods as pi
#vecs is a list of vectors of the embeddings
vecs = models[0].wv.vectors
#vocab is a list of the words
vocab = list(models[0].wv.vocab.keys())

###########################################
# DEFINING EMBEDDING                      #
###########################################
NY_Embed = embedding.Embedding(vecs, vocab)

###########################################
# DEFINING POSITIVE-NEGATIVE SEEDS        #
###########################################
positive_seed = list(enriched_lexicon[enriched_lexicon["Labels"] == 1]["Words"])[:50]
negative_seed = list(enriched_lexicon[enriched_lexicon["Labels"] == -1]["Words"])

In [ ]:
pi.random_walk(NY_Embed, positive_seed, negative_seed)